<a href="https://colab.research.google.com/github/LaJeremi/Tensorflow-nlp-tutorial-Practice-/blob/main/18.%20Fine-tuning%20BERT%20(Cls%2C%20NER%2C%20NLI)/%2018_08_sentence_transformer_chatbot_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 18-08 BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇

 https://wikidocs.net/154530

In [ ]:
pip install sentence_transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f77ff406e701fac4b7c5a5ff77e0789bde30ee6cd198021886e5e8c868c2a247
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [ ]:

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()
     

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')


In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)


In [ ]:
train_data


,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.20179586, -0.03443814, 1.5395721, 0.0106974..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.07716598, -0.0342782, 0.86244255, 0.0263605..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.10445245, -0.012432207, 1.0132878, 0.022501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760746, -0.046716847, 0.893694, 0.0210474..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002865, 0.031961147, 1.4915428, 4.343475..."
...,...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[0.15363297, -0.32605597, 0.6987676, 0.1091406..."
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[0.15363297, -0.32605597, 0.6987676, 0.1091406..."
11820,흑기사 해주는 짝남.,설렜겠어요.,2,"[-0.036085084, -0.0052650566, 0.89135355, -0.0..."
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[-0.08468656, -0.14302962, 0.11526109, 0.38679..."


In [ ]:

import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))
     

In [ ]:

def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']
     

In [ ]:
return_similar_answer('결혼하고싶어')


'좋은 사람이랑 결혼할 수 있을 거예요.'

In [ ]:
return_similar_answer('나랑 커피먹을래?')


'카페인이 필요한 시간인가 봐요.'

In [ ]:
return_similar_answer('반가워')


'저도 반가워요.'

In [ ]:
return_similar_answer('사랑해')


'상대방에게 전해보세요.'

In [ ]:
return_similar_answer('너는 누구니?')


'저는 위로봇입니다.'

In [ ]:
return_similar_answer('영화')


'저도 영화 보여주세요.'

In [ ]:
return_similar_answer('너무 짜증나')


'짜증날 땐 짜장면'

In [ ]:
return_similar_answer('화가납니다')


'화를 참는 연습을 해보세요.'

In [ ]:
return_similar_answer('나랑 놀자')


'지금 그러고 있어요.'

In [ ]:
return_similar_answer('나랑 게임하자')


'같이 놀아요.'

In [ ]:
return_similar_answer('출근하기싫어')


'씻고 푹 쉬세요.'

In [ ]:
return_similar_answer('여행가고싶다')


'이김에 떠나보세요.'

In [ ]:
return_similar_answer('너 말 잘한다')


'그런 사람이 있으면 저 좀 소개시켜주세요.'